## Visualizations

In [24]:
%useLatestDescriptors
%use dataframe
%use kandy

In [25]:
import java.nio.file.Path
import kotlin.io.path.*

fun DataFrame.Companion.readMetricsJson(json: Path): DataFrame<Int> {
    val metricsFile = json / "output/metrics.json"
    
    return when {
        metricsFile.exists() -> readJson(metricsFile.pathString).cast<Int>()
        else -> emptyDataFrame()
    } 
}

fun Path.readAllInstances() = toFile()
    .walk()
    .filter { it.isDirectory && (it.name.startsWith("f") || it.name.startsWith("v")) }
    .map {
        it.toPath()
            .listDirectoryEntries()
            .map { DataFrame.readMetricsJson(it) }
            .reduce { acc, dataFrame -> acc.concat(dataFrame) }
    }
    .reduce { acc, df -> acc.concat(df) }

fun <T> DataFrame<T>.groupByInstanceName() = groupBy {
    val instanceNames = column<String>("InputDir").map { it.removePrefix("study/instances/").split("/", limit = 2) }
    (instanceNames.map { it[0] } named "severity") and (instanceNames.map { it[1].removePrefix("issue_").replace("_inlined", "") } named "issue")
}

fun <T> DataFrame<T>.aggregateTotalAfterSize(toolName: String) = groupByInstanceName()
    .aggregate { get("Total").get("AfterSize").cast<Int>().mean() into toolName }
    .gather(toolName).into("Tool", "Size")


In [26]:
val seruPersesV1 = Path("results_perses_second_changedLoop").readAllInstances()
val seruPersesV2 = Path("results_perses_wrong_interpolation").readAllInstances()
val seruPersesV3 = Path("results_perses_third_no_recover").readAllInstances()
val seruPerses = Path("results_perses").readAllInstances()

val seruVulcanV2 = Path("results_vulcan_wrong_interpolation").readAllInstances()
val seruVulcanV3 = Path("results_vulcan_third_no_recover").readAllInstances()
val seruVulcan = Path("results_vulcan").readAllInstances()

In [27]:
val groupedInstanceDfs = seruPerses.groupByInstanceName()

In [34]:
val totalSizeDf = groupedInstanceDfs
    .aggregate {
        Total.BeforeSize.max() into "Before"
        Total.AfterSize.mean() into "Seru+Perses"
    }
    .gather("Before", "Seru+Perses").into("Tool", "Size")
//    .concat(seruPersesV1.aggregateTotalAfterSize("Seru+Perses v1"))
//    .concat(seruPersesV2.aggregateTotalAfterSize("Seru+Perses v2"))
//    .concat(seruPersesV3.aggregateTotalAfterSize("Seru+Perses v3"))
    .concat(seruVulcan.aggregateTotalAfterSize("Seru+Vulcan"))
//    .concat(seruVulcanV2.aggregateTotalAfterSize("Seru+Vulcan v2"))
//    .concat(seruVulcanV3.aggregateTotalAfterSize("Seru+Vulcan v3"))
totalSizeDf

severity,issue,Tool,Size
extra,2/final,Before,249
extra,2/final,Seru+Perses,75.000000
panic,2584/v1,Before,251
panic,2584/v1,Seru+Perses,42.200000
panic,2584/v2,Before,106
panic,2584/v2,Seru+Perses,29.000000
panic,2584/final,Before,25
panic,2584/final,Seru+Perses,25.000000
panic,2490/v1,Before,420
panic,2490/v1,Seru+Perses,85.000000


In [35]:
totalSizeDf.plot {
    x(issue) {
        axis.name = "Instance"
    }
    bars {
        y(Size)
        fillColor(Tool)
    }
    layout {
        size = 900 to 500
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="rJsifh"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"issue":["2/final","2/final","2584/v1","2584/v1","2584/v2","2584/v2","2584/final","2584/final","2490/v1","2490/v1","2490/v2","2490/v2","2490/final","2490/final","2218/v1","2218/v1","2218/v2","2218/v2","2218/final","2218/final","2246/v1","2246/v1","2246/final","2246/final","2473/v1","2473/v1","2473/final","2473/final","2209/v1","2209/v1","2209/final","2209/final","2/final","2584/v1","2584/v2","2584/final","2490/v2","2490/final","2218/v1","2218/v2","2218/final","2246/final","2473/v1","2473/final","2209/final"],
"Size":[249.0,75.0,251.0,42.2,106.0,29.0,25.0,25.0,420.0,85.0,67.0,44.0,48.0,42.0,63.0,47.0,39.0,29.0,37.0,29.0,110.0,45.0,34.0,30.0,67.0,36.0,35.0,30.0,274.0,134.0,124.0,83.0,77.0,26.0,31.0,25.0,49.0,42.0,37.0,29.0,29.0,26.0,28.0,30.0,35.0],
"Tool":["Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Before","Seru+Perses","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan","Seru+Vulcan"]
},
"ggsize":{
"width":900.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Instance"
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"issue",
"y":"Size",
"fill":"Tool"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("rJsifh");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2/final 
 
 
 
 
 
 
 
 
 2584/v1 
 
 
 
 
 
 
 
 
 2584/v2 
 
 
 
 
 
 
 
 
 2584/final 
 
 
 
 
 
 
 
 
 2490/v1 
 
 
 
 
 
 
 
 
 2490/v2 
 
 
 
 
 
 
 
 
 2490/final 
 
 
 
 
 
 
 
 
 2218/v1 
 
 
 
 
 
 
 
 
 2218/v2 
 
 
 
 
 
 
 
 
 2218/final 
 
 
 
 
 
 
 
 
 2246/v1 
 
 
 
 
 
 
 
 
 2246/final 
 
 
 
 
 
 
 
 
 2473/v1 
 
 
 
 
 
 
 
 
 2473/final 
 
 
 
 
 
 
 
 
 2209/v1 
 
 
 
 
 
 
 
 
 2209/final 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 250 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 350 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 Size 
 
 
 
 
 Instance 
 
 
 
 
 
 
 
 
 Tool 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Before 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Seru+Perses 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Seru+Vulcan